<a href="https://colab.research.google.com/github/Cairo-Henrique/Movie-Recommender/blob/main/train_movies_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown 153zyDsohGQ8pk_CzI8cHHnSIiw1jOohe

Downloading...
From: https://drive.google.com/uc?id=153zyDsohGQ8pk_CzI8cHHnSIiw1jOohe
To: /content/movies_with_embeddings.pkl
100% 37.5M/37.5M [00:00<00:00, 63.5MB/s]


In [ ]:
import pandas as pd
import os
from pathlib import Path

from sklearn.metrics.pairwise import cosine_similarity

from transformers import BertTokenizer, BertModel
import torch

In [ ]:
def recommend_movies(input_embedding, movies, top_n=5):
    # Calcular similaridade de cosseno entre o input e os embeddings dos filmes
    similarities = cosine_similarity([input_embedding], movie_embeddings)[0]

    # Criar uma cópia do DataFrame original para não modificar os dados originais
    movies_copy = movies.copy()

    # Adicionar similaridades ao DataFrame
    movies_copy['similarity'] = similarities

    # Excluir filmes sem overview
    movies_copy = movies_copy[movies_copy['overview'].str.strip() != ""]

    # Ordenar pelos mais similares e retornar os top_n
    recommended = movies_copy.sort_values(by='similarity', ascending=False).head(top_n)

    return recommended[['title', 'overview', 'similarity']]

In [ ]:
def get_movie_embedding(title, overview, genres):
    # Gera embedding usando BERT
    text = f"title: {title}. overview: {overview}. genres: {genres}"
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    return embedding

In [ ]:
# Carregar modelo BERT pré-treinado
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
# Caminho absoluto do arquivo CSV corrigido no ambiente Colab
movies_path = '/content/movies_fixed.csv' # Updated path

# Leitura do arquivo
movies = pd.read_csv(movies_path, encoding='utf-8')
#movies = movies.head(100)

# Garantir que todas as colunas existem e estão tratadas
movies['title'] = movies['title'].fillna('').astype(str)
movies['overview'] = movies['overview'].fillna('').astype(str)
movies['genres'] = movies['genres'].fillna('').astype(str)

# Gerar embeddings considerando title + overview + genres
movies['embedding'] = movies.apply(
    lambda row: get_movie_embedding(row['title'], row['overview'], row['genres']),
    axis=1
)

movie_embeddings = list(movies['embedding'])

# Input do usuário
input_description = input("Enter a movie description: ")
input_embedding = get_movie_embedding(input_description, input_description, input_description)

# Obter recomendações
recommended_movies = recommend_movies(input_embedding, movies, top_n=5)

print("Recommended Movies:")
print(recommended_movies)

FileNotFoundError: [Errno 2] No such file or directory: '/content/movies_fixed.csv'

In [ ]:
# Salva embeddings
movies.to_pickle("/content/movies_with_embeddings.pkl")

In [ ]:
import pandas as pd

movies = pd.read_pickle("/content/movies_with_embeddings.pkl")
movie_embeddings = list(movies['embedding'])

# Input do usuário
input_description = input("Enter a movie description: ")
input_embedding = get_movie_embedding(input_description, input_description, input_description)

def recommend_movies(input_embedding, movies, top_n=5):
    # Calcular similaridade de cosseno entre o input e os embeddings dos filmes
    similarities = cosine_similarity([input_embedding], movie_embeddings)[0]

    # Criar uma cópia do DataFrame original para não modificar os dados originais
    movies_copy = movies.copy()

    # Adicionar similaridades ao DataFrame
    movies_copy['similarity'] = similarities

    # Excluir filmes sem overview
    movies_copy = movies_copy[movies_copy['overview'].str.strip() != ""]

    # Excluir filmes sem gender
    movies_copy = movies_copy[movies_copy['genres'].str.strip() != ""]

    # Ordenar pelos mais similares e retornar os top_n
    recommended = movies_copy.sort_values(by='similarity', ascending=False).head(top_n)

    return recommended[['title', 'overview', 'similarity']]

# Obter recomendações
recommended_movies = recommend_movies(input_embedding, movies, top_n=5)

print("Recommended Movies:")
print(recommended_movies)

Enter a movie description: central station
Recommended Movies:
                                                title  \
3785                               extremely easy job   
2674             the monster at the end of this story   
3494                one piece: strong world episode 0   
4255  love, chunibyo & other delusions! rikka version   
6852                          porno: situación límite   

                                               overview  similarity  
3785              remake from korean movie extreme job.    0.775866  
2674  the monster at the end of this story is sesame...    0.765009  
3494  the introductory chapter to the movie, strong ...    0.764093  
4255  summary of the first season of the show, as se...    0.759645  
6852    classic and very little known 80s erotic movie.    0.756727  


In [ ]:
# Display the titles and overviews of the recommended movies
import pandas as pd

movies_fixed = pd.read_csv('/content/movies_fixed.csv')

for movie_index in [2010, 644, 676, 7791, 2647]:
  # Access the row using integer-location based indexing (.iloc)
  movie_info = movies_fixed.iloc[movie_index]
  print(f"Title: {movie_info['title']}")
  print(f"Overview: {movie_info['overview']}")
  print("-" * 30) # Separator for readability


Title: the king's avatar: for the glory
Overview: prequel movie to the animated series the king's avatar, dealing with ye xiu's original entry into the pro gaming world of glory, and the first pro league series tournament.
------------------------------
Title: fast x
Overview: the tenth installment in the fast saga.
------------------------------
Title: fate/grand order: the movie – divine realm of the round table: camelot – paladin; agateram
Overview: part two of fate/grand order: shinsei entaku ryouiki camelot - wandering; agateram; an adaptation of the the sixth holy grail war, the sacred round table realm camelot singularity of fate/grand order.
------------------------------
Title: macross: do you remember love?
Overview: it is a.d. 2009 and the human race is caught in a war between giant humanoids, male zentrans and female meltrans. returning from the edge of our solar system after making a space fold, the sdf-1 macross makes the long journey back to earth with survivors of south